In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
def loadpm():
    idx=1
    idx=str(idx)
    imgpath='pics/full/40x40/poke'+idx+'.jpg'
    try:
        pmimg=plt.imread(imgpath)
        pmimg=pmimg.reshape(1,pmimg.shape[0], pmimg.shape[1], 3)
    except:
        return
    idx=int(idx)
    idx+=1
    while True:
        idx=str(idx)
        imgpath='pics/full/40x40/poke'+idx+'.jpg'
        
        try:
            img=plt.imread(imgpath)
        except:
            break
        pmimg =np.vstack((pmimg,img.reshape(1,img.shape[0], img.shape[1], 3)))
        idx=int(idx)
        idx+=1
    return pmimg

In [2]:
pokemon=loadpm()
pokemon=(pokemon-127.5)/127.5
print((pokemon.shape))

(840, 40, 40, 3)


In [3]:
import os
import numpy as np
import tensorflow as tf
from IPython.core.debugger import Tracer
from keras.layers import Input, Dense, Reshape, Flatten, Dropout,Conv2D,MaxPooling2D,Conv2DTranspose
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential,Model
from keras.optimizers import Adam,RMSprop
import keras.backend as K
from keras.engine.topology import Layer
import matplotlib.pyplot as plt
from keras.layers.merge import _Merge

class Subtract(_Merge):
    def _merge_function(self, inputs):
        output = inputs[0]
        for i in range(1, len(inputs)):
            output = output-inputs[i]
        return output
class GradNorm(Layer):
    def __init__(self, **kwargs):
        super(GradNorm, self).__init__(**kwargs)

    def build(self, input_shapes):
        super(GradNorm, self).build(input_shapes)

    def call(self, inputs):
        target, wrt = inputs
        grads = K.gradients(target, wrt)
        assert len(grads) == 1
        grad = grads[0]
        return K.sqrt(K.sum(K.batch_flatten(K.square(grad)), axis=1, keepdims=True))

    def compute_output_shape(self, input_shapes):
        return (input_shapes[1][0], 1)

Using TensorFlow backend.


In [4]:
class Generator():
    def __init__(self,gen_dim,output_size):
        self.network=Sequential()
        element=1
        for i in output_size:
            element*=i
        
        self.network.add(Dense(element//16*256,input_dim=gen_dim,activation='relu'))
        self.network.add(Reshape((output_size[0]//4,output_size[1]//4,output_size[2]*256)))

        #self.network.add(LeakyReLU(0.2))
        
        
        self.network.add(Conv2DTranspose(filters=256,kernel_size=(3,3),strides=2,padding='same',activation='relu'))

        #self.network.add(LeakyReLU(0.2))
        self.network.add(Conv2DTranspose(filters=256,kernel_size=(3,3),strides=2,padding='same',activation='relu'))

        #self.network.add(LeakyReLU(0.2))
        self.network.add(Conv2DTranspose(filters=3,kernel_size=(3,3),padding='same',activation='tanh'))
        self.network.summary()
class Discriminator():
    #Critic
    def __init__(self,input_size):
        self.network=Sequential()
        element=1
        for i in input_size:
            element*=i

        self.network.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',input_shape=input_size))
        self.network.add(LeakyReLU(0.2))
        self.network.add(Conv2D(filters=128,kernel_size=(3,3),padding='same'))
        self.network.add(LeakyReLU(0.2))
        self.network.add(Conv2D(filters=256,kernel_size=(3,3),padding='same'))
        self.network.add(LeakyReLU(0.2))
        self.network.add(Conv2D(filters=256,kernel_size=(3,3),padding='same'))
        self.network.add(LeakyReLU(0.2))
        
        self.network.add((Flatten()))
        #self.network.add(Dropout(0.2))
        #self.network.add(Dense(30))
        self.network.add(Dense(1,activation='linear'))
        
        self.network.summary()
        
        
        
class GAN():
    def __init__(self,trainingset,gen_dim):
        self.trainshape=trainingset.shape
        self.trainingset=trainingset
        self.gen_dim=gen_dim
        
        self.n_discriminator = 8
        
        self._lambda = 10
        optimizer = Adam(lr=0.0001,beta_1=0,beta_2=0.9)#RMSprop(lr=0.00005)
        #gen_dis
        self.generator=Generator(gen_dim,(trainingset.shape[1],trainingset.shape[2],trainingset.shape[3]))
        self.discriminator=Discriminator((trainingset.shape[1],trainingset.shape[2],trainingset.shape[3]))
        self.GAN_input=Input((gen_dim,))
        self.GAN_output=self.discriminator.network(self.generator.network(self.GAN_input))
        self.network=Model(self.GAN_input,self.GAN_output)
        self.network.compile(loss=self.wasserstein_loss,optimizer=optimizer)
        
        #gradient
        self.inputshape=self.trainshape[1:]
        self.real_input,self.gen_input,self.mixed_input=Input(self.inputshape),Input(self.inputshape),Input(self.inputshape)
        
        self.sub=Subtract()([self.discriminator.network(self.gen_input),self.discriminator.network(self.real_input)])
        self.grad_norm=GradNorm()([self.discriminator.network(self.mixed_input),self.mixed_input])
        self.grad=Model([self.gen_input,self.real_input,self.mixed_input],[self.sub,self.grad_norm])
        self.grad.compile(optimizer=optimizer,loss=[self.mean_loss,'MSE'],loss_weights=[1,self._lambda])
    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)
    def mean_loss(self, y_true, y_pred):
        return K.mean(y_pred)
    def train(self,iterations):
        self.batch_size=20
        self.real_label=np.ones((self.batch_size, 1))
        self.gen_label=-np.ones((self.batch_size, 1))
        for step in range(iterations):
        #train discriminator
            for i in range(self.n_discriminator):
            #prepare img
                self.random_gen_vectors=np.random.normal(size=(self.batch_size,self.gen_dim))
                self.gen_img=self.generator.network.predict(self.random_gen_vectors)
                self.epsilon=np.random.uniform(0,1,size=(self.batch_size,1,1,1))
                self.idx=np.random.randint(0,self.trainingset.shape[0],self.batch_size)
                self.real_img=self.trainingset[self.idx]
                self.mixed=self.epsilon*self.real_img+(1-self.epsilon)*self.gen_img
                
        
                #self.labels -= 0.1 * np.random.rand(self.labels.shape[0],self.labels.shape[1])
                self.discriminator.network.trainable = True
                self.d_loss = self.grad.train_on_batch([self.gen_img,self.real_img,self.mixed],[np.ones((self.batch_size,1)),np.ones((self.batch_size,1))])
                
                
        #train generator
            self.discriminator.network.trainable = False
            self.random_gen_vector=np.random.normal(size=(self.batch_size,self.gen_dim))
            self.g_loss=self.network.train_on_batch(self.random_gen_vector,self.gen_label)
            
            print('iter:%d,w_dis=%f,g_loss=%f'%(step,-self.d_loss[0],self.g_loss))
            if step%50==0:
                #save image
                random_vector=np.random.normal(size=(25,self.gen_dim))
                images=self.generator.network.predict(random_vector)
                images=(images+1)*127.5
                plt.figure(figsize=(10, 10))
                for i in range(images.shape[0]):
                    plt.subplot(5,5,i+1)
                    image = images[i, :, :, :]
                    image = np.reshape(image,(self.trainshape[1:]))
                    plt.imshow(image.astype(np.uint8))
                    plt.axis('off')
                plt.tight_layout()
                plt.savefig("output/poke_%d.png" % (step+0))
                plt.close('all')
            if step%500==0:
                gan.network.save('pokemon_model.h5')
                gan.grad.save('pokemon_grad.h5')
                    
gan=GAN(pokemon,25)
#gan.network.load_weights('eevee_weights.h5')
#gan.grad.load_weights('eevee_grad_weights.h5')
gan.train(20000)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 76800)             1996800   
_________________________________________________________________
reshape_1 (Reshape)          (None, 10, 10, 768)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 20, 20, 256)       1769728   
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 40, 40, 256)       590080    
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 40, 40, 3)         6915      
Total params: 4,363,523
Trainable params: 4,363,523
Non-trainable params: 0
_________________________________________________________________
____________________________________________________________

C:\Users\Rita\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


iter:0,w_dis=103.333809,g_loss=-0.300709
iter:1,w_dis=136.612732,g_loss=-3.122970
iter:2,w_dis=135.435120,g_loss=-5.743940
iter:3,w_dis=132.224915,g_loss=-9.575566
iter:4,w_dis=125.801895,g_loss=-15.804645
iter:5,w_dis=117.774155,g_loss=-24.076656
iter:6,w_dis=108.447403,g_loss=-31.942343
iter:7,w_dis=99.457016,g_loss=-40.563606
iter:8,w_dis=91.780434,g_loss=-50.824623
iter:9,w_dis=77.183594,g_loss=-55.289490
iter:10,w_dis=73.586433,g_loss=-62.974659
iter:11,w_dis=64.512253,g_loss=-66.879776
iter:12,w_dis=59.107788,g_loss=-68.843727
iter:13,w_dis=50.511841,g_loss=-63.218651
iter:14,w_dis=53.568909,g_loss=-65.000053
iter:15,w_dis=49.949661,g_loss=-61.714100
iter:16,w_dis=52.325417,g_loss=-60.074085
iter:17,w_dis=43.872902,g_loss=-48.645020
iter:18,w_dis=47.317986,g_loss=-53.026764
iter:19,w_dis=44.542751,g_loss=-48.450726
iter:20,w_dis=52.450684,g_loss=-31.110056
iter:21,w_dis=42.547096,g_loss=-48.491558
iter:22,w_dis=41.653870,g_loss=-28.943630
iter:23,w_dis=38.506924,g_loss=-26.776300

iter:194,w_dis=16.019165,g_loss=-43.364376
iter:195,w_dis=13.081602,g_loss=-39.358112
iter:196,w_dis=13.739755,g_loss=-38.760475
iter:197,w_dis=13.177546,g_loss=-34.090538
iter:198,w_dis=14.392443,g_loss=-37.560745
iter:199,w_dis=15.232458,g_loss=-48.421562
iter:200,w_dis=14.927546,g_loss=-48.210251
iter:201,w_dis=15.538883,g_loss=-41.055988
iter:202,w_dis=14.633474,g_loss=-42.049286
iter:203,w_dis=14.836427,g_loss=-52.111755
iter:204,w_dis=12.374880,g_loss=-37.975174
iter:205,w_dis=14.959847,g_loss=-40.095039
iter:206,w_dis=13.547248,g_loss=-37.113091
iter:207,w_dis=15.246840,g_loss=-45.015892
iter:208,w_dis=14.128638,g_loss=-46.797417
iter:209,w_dis=14.751512,g_loss=-38.959564
iter:210,w_dis=13.981634,g_loss=-43.106689
iter:211,w_dis=13.763659,g_loss=-41.197636
iter:212,w_dis=14.737873,g_loss=-40.470009
iter:213,w_dis=16.504272,g_loss=-48.092827
iter:214,w_dis=16.736103,g_loss=-52.687977
iter:215,w_dis=13.484574,g_loss=-37.220497
iter:216,w_dis=15.289354,g_loss=-37.158592
iter:217,w_

iter:385,w_dis=11.317764,g_loss=-56.114918
iter:386,w_dis=12.977627,g_loss=-57.778332
iter:387,w_dis=13.567617,g_loss=-66.334488
iter:388,w_dis=13.150120,g_loss=-56.308861
iter:389,w_dis=13.032940,g_loss=-53.740883
iter:390,w_dis=13.549736,g_loss=-57.687492
iter:391,w_dis=15.043883,g_loss=-53.477776
iter:392,w_dis=12.448776,g_loss=-56.843498
iter:393,w_dis=14.286260,g_loss=-59.510509
iter:394,w_dis=14.173061,g_loss=-66.012436
iter:395,w_dis=12.332879,g_loss=-68.349846
iter:396,w_dis=13.582352,g_loss=-58.936676
iter:397,w_dis=13.421850,g_loss=-69.329964
iter:398,w_dis=13.039907,g_loss=-66.831627
iter:399,w_dis=12.651089,g_loss=-63.287464
iter:400,w_dis=13.234834,g_loss=-66.170937
iter:401,w_dis=14.659181,g_loss=-60.922985
iter:402,w_dis=13.410638,g_loss=-66.050842
iter:403,w_dis=13.644493,g_loss=-63.489887
iter:404,w_dis=12.507474,g_loss=-62.246620
iter:405,w_dis=13.161223,g_loss=-58.388073
iter:406,w_dis=13.194083,g_loss=-63.608788
iter:407,w_dis=12.543109,g_loss=-65.893654
iter:408,w_

...

iter:6311,w_dis=1.085673,g_loss=-3960.987549
iter:6312,w_dis=1.176125,g_loss=-3533.468750
iter:6313,w_dis=2.807838,g_loss=-3608.145996
iter:6314,w_dis=1.077897,g_loss=-4096.291992
iter:6315,w_dis=2.303926,g_loss=-5053.831543
iter:6316,w_dis=3.621958,g_loss=-4420.265625
iter:6317,w_dis=0.655479,g_loss=-3573.783936
iter:6318,w_dis=1.104345,g_loss=-4184.478516
iter:6319,w_dis=3.063939,g_loss=-4646.563477
iter:6320,w_dis=0.095830,g_loss=-4001.071533
iter:6321,w_dis=1.231873,g_loss=-4322.429199
iter:6322,w_dis=0.774424,g_loss=-4391.964844
iter:6323,w_dis=-0.245451,g_loss=-4287.467285
iter:6324,w_dis=0.427223,g_loss=-3967.169189
iter:6325,w_dis=-0.206449,g_loss=-3823.365234
iter:6326,w_dis=0.338006,g_loss=-3380.403809
iter:6327,w_dis=0.012185,g_loss=-3175.102783
iter:6328,w_dis=1.416792,g_loss=-4077.727295
iter:6329,w_dis=0.529721,g_loss=-2932.604980
iter:6330,w_dis=3.199235,g_loss=-3315.078857
iter:6331,w_dis=1.121484,g_loss=-3236.781738
iter:6332,w_dis=2.339772,g_loss=-4084.845703
iter:633

iter:6493,w_dis=3.812435,g_loss=-5623.011719
iter:6494,w_dis=-1.478780,g_loss=-5223.473145
iter:6495,w_dis=2.098516,g_loss=-5868.165039
iter:6496,w_dis=0.274869,g_loss=-4535.539062
iter:6497,w_dis=0.482235,g_loss=-5451.183594
iter:6498,w_dis=0.310147,g_loss=-4968.450684
iter:6499,w_dis=0.823137,g_loss=-4161.487305
iter:6500,w_dis=1.499954,g_loss=-5038.570312
iter:6501,w_dis=0.585202,g_loss=-5370.864746
iter:6502,w_dis=1.682911,g_loss=-4223.484863
iter:6503,w_dis=0.225468,g_loss=-5092.691895
iter:6504,w_dis=3.208612,g_loss=-5217.248047
iter:6505,w_dis=1.014519,g_loss=-4362.100098
iter:6506,w_dis=0.618179,g_loss=-4612.150879
iter:6507,w_dis=2.066966,g_loss=-5731.520508
iter:6508,w_dis=-3.428717,g_loss=-4824.249023
iter:6509,w_dis=0.441848,g_loss=-4353.820312
iter:6510,w_dis=3.096650,g_loss=-5848.717773
iter:6511,w_dis=1.431290,g_loss=-6044.293945
iter:6512,w_dis=0.538795,g_loss=-4716.880859
iter:6513,w_dis=-4.645350,g_loss=-4973.328125
iter:6514,w_dis=2.643850,g_loss=-5652.713379
iter:65

iter:6674,w_dis=-0.286982,g_loss=-5608.735352
iter:6675,w_dis=1.715714,g_loss=-5507.016602
iter:6676,w_dis=1.806333,g_loss=-4926.987305
iter:6677,w_dis=-1.022935,g_loss=-4831.583008
iter:6678,w_dis=0.520740,g_loss=-4888.054199
iter:6679,w_dis=-0.709676,g_loss=-5957.696289
iter:6680,w_dis=-2.334985,g_loss=-5591.325684
iter:6681,w_dis=0.489423,g_loss=-5879.536621
iter:6682,w_dis=0.894154,g_loss=-5544.665039
iter:6683,w_dis=0.545594,g_loss=-4956.676758
iter:6684,w_dis=0.487203,g_loss=-5825.882812
iter:6685,w_dis=0.139352,g_loss=-6335.334473
iter:6686,w_dis=1.733899,g_loss=-5187.454102
iter:6687,w_dis=2.018698,g_loss=-6074.258301
iter:6688,w_dis=-0.813085,g_loss=-5406.266602
iter:6689,w_dis=-0.990230,g_loss=-5760.696777
iter:6690,w_dis=1.419465,g_loss=-6174.979492
iter:6691,w_dis=2.226551,g_loss=-6481.500977
iter:6692,w_dis=12.748892,g_loss=-6419.688965
iter:6693,w_dis=-0.512854,g_loss=-4685.528320
iter:6694,w_dis=-0.350916,g_loss=-5359.294434
iter:6695,w_dis=0.291469,g_loss=-6367.717773
i

iter:6855,w_dis=0.266900,g_loss=-5951.709961
iter:6856,w_dis=0.443618,g_loss=-5725.770508
iter:6857,w_dis=1.698974,g_loss=-5303.148438
iter:6858,w_dis=1.212009,g_loss=-5780.163574
iter:6859,w_dis=0.233869,g_loss=-5729.091309
iter:6860,w_dis=3.751718,g_loss=-5461.233398
iter:6861,w_dis=0.533652,g_loss=-5156.175293
iter:6862,w_dis=-10.483741,g_loss=-3968.671875
iter:6863,w_dis=1.289680,g_loss=-5603.443848
iter:6864,w_dis=0.715002,g_loss=-5719.107910
iter:6865,w_dis=0.640245,g_loss=-6410.744629
iter:6866,w_dis=0.632434,g_loss=-5022.550781
iter:6867,w_dis=1.603405,g_loss=-5764.989258
iter:6868,w_dis=1.153253,g_loss=-5408.484375
iter:6869,w_dis=0.236793,g_loss=-4832.819824
iter:6870,w_dis=0.381003,g_loss=-5818.532227
iter:6871,w_dis=1.200565,g_loss=-4843.158203
iter:6872,w_dis=0.721037,g_loss=-5828.506348
iter:6873,w_dis=0.227362,g_loss=-4956.847656
iter:6874,w_dis=0.894721,g_loss=-5887.541992
iter:6875,w_dis=0.397027,g_loss=-5890.680664
iter:6876,w_dis=0.273740,g_loss=-6274.295410
iter:687

KeyboardInterrupt: 

In [5]:
from keras.models import load_model
#gan.network.save('eevee_model.h5')
#gan.grad.save('eevee_grad.h5')
gan.network.save_weights('pokemon_weights3.h5')
gan.grad.save_weights('pokemon_grad_weights3.h5')
'''
def wasserstein_loss(y_true, y_pred):
        return K.mean(y_true * y_pred)
def mean_loss(y_true, y_pred):
    return K.mean(y_pred)
optimizer = Adam(lr=0.0001,beta_1=0,beta_2=0.9)

gan.network=load_model('dragon_model.h5', custom_objects={'wasserstein_loss': wasserstein_loss})
gan.grad=load_model('dragon_grad.h5', custom_objects={'mean_loss': mean_loss,'GradNorm':GradNorm,'Subtract':Subtract,'optimizer':optimizer})
gan.generator.network=gan.network.layers[1]
gan.discriminator.network=gan.network.layers[2]
'''

"\ndef wasserstein_loss(y_true, y_pred):\n        return K.mean(y_true * y_pred)\ndef mean_loss(y_true, y_pred):\n    return K.mean(y_pred)\noptimizer = Adam(lr=0.0001,beta_1=0,beta_2=0.9)\n\ngan.network=load_model('dragon_model.h5', custom_objects={'wasserstein_loss': wasserstein_loss})\ngan.grad=load_model('dragon_grad.h5', custom_objects={'mean_loss': mean_loss,'GradNorm':GradNorm,'Subtract':Subtract,'optimizer':optimizer})\ngan.generator.network=gan.network.layers[1]\ngan.discriminator.network=gan.network.layers[2]\n"

In [ ]:

gan.network.compile(loss=gan.wasserstein_loss,optimizer=optimizer)
gan.grad.compile(optimizer=optimizer,loss=[gan.mean_loss,'MSE'],loss_weights=[1,gan._lambda])

gan.train(20000)

C:\Users\Rita\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


iter:0,w_dis=19.838118,g_loss=-3955.755371
iter:1,w_dis=12.912529,g_loss=-6526.853027
iter:2,w_dis=17.649376,g_loss=-10306.881836
iter:3,w_dis=34.334625,g_loss=-15923.833984
iter:4,w_dis=40.501049,g_loss=-24054.396484
iter:5,w_dis=43.300419,g_loss=-35008.699219
iter:6,w_dis=35.517178,g_loss=-49418.324219
iter:7,w_dis=42.377281,g_loss=-67969.203125
iter:8,w_dis=37.149498,g_loss=-91503.265625
iter:9,w_dis=30.578796,g_loss=-120498.578125
iter:10,w_dis=48.846802,g_loss=-155898.109375
iter:11,w_dis=51.985214,g_loss=-198645.156250
iter:12,w_dis=66.121971,g_loss=-249289.703125
iter:13,w_dis=52.579262,g_loss=-309091.437500
iter:14,w_dis=40.415405,g_loss=-378256.281250
iter:15,w_dis=58.308144,g_loss=-458956.593750
iter:16,w_dis=44.633926,g_loss=-550760.437500
iter:17,w_dis=43.842243,g_loss=-654001.000000
iter:18,w_dis=55.952801,g_loss=-773802.687500
iter:19,w_dis=48.165154,g_loss=-906877.375000
iter:20,w_dis=56.510994,g_loss=-1054731.750000
iter:21,w_dis=35.243332,g_loss=-1221684.375000
iter:22